# Groups, joins, and pivots #

In [2]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Classifying movies ##

In [ ]:
top_movies = Table.read_table('top_movies_2017.csv')
top_movies

In [ ]:
top_movies.group('Studio')

In [ ]:
# The following code finds the gross revenue in total
# over all movies released by a studio. Unpack this code 
# and understand it before moving on to the next part!

top_movies.select('Studio', 'Gross').group('Studio', collect=sum)

In [ ]:
# Exercise: 
# Make a scatter plot of the count of top movies released by year, 
# counting only movies released after 2000.
...

In [ ]:
# More challenging: We're going to make a scatter chart 
# of the adjusted gross for the TOP movie released in each year.

# Make a table which ONLY has columns for year and adjusted gross.
small_table = ...

# Make a table which groups things by year and uses COLLECT
# to find the highest grossing movie that year.
highest_grossing_by_year = ...

## Cross-Classifying

In [3]:
# Load ice cream data
cones = Table.read_table('cones.csv')
cones

Flavor,Color,Price
strawberry,pink,3.55
chocolate,light brown,4.75
chocolate,dark brown,5.25
strawberry,pink,5.25
chocolate,dark brown,5.25
bubblegum,pink,4.75


In [6]:
# Group the cones by flavor, color, and both:
#cones.group('Flavor').show()
#cones.group('Color').show()

# Try to run this line and it goes wrong!
# cones.group('Flavor', 'Color').show()

# Use this one instead:
#cones.group(['Flavor', 'Color']).show()
#cones.group(['Color', 'Flavor']).show()
Table.group?

Signature: Table.group(self, column_or_label, collect=None)
Docstring:
Group rows by unique values in a column; count or aggregate others.

Args:
    ``column_or_label``: values to group (column label or index, or array)

    ``collect``: a function applied to values in other columns for each group

Returns:
    A Table with each row corresponding to a unique value in ``column_or_label``,
    where the first column contains the unique values from ``column_or_label``, and the
    second contains counts for each of the unique values. If ``collect`` is
    provided, a Table is returned with all original columns, each containing values
    calculated by first grouping rows according to ``column_or_label``, then applying
    ``collect`` to each set of grouped values in the other columns.

Note:
    The grouped column will appear first in the result table. If ``collect`` does not
    accept arguments with one of the column types, that column will be empty in the resulting
    table.

>>> mar

In [8]:
# We can use a pivot table to organize this data differently
# by listing all possible combinations:
cones.pivot('Flavor', 'Color')
Table.pivot?

Signature: Table.pivot(self, columns, rows, values=None, collect=None, zero=None)
Docstring:
Generate a table with a column for each unique value in ``columns``,
with rows for each unique value in ``rows``. Each row counts/aggregates
the values that match both row and column based on ``collect``.

Args:
    ``columns`` -- a single column label or index, (``str`` or ``int``),
        used to create new columns, based on its unique values.
    ``rows`` -- row labels or indices, (``str`` or ``int`` or list),
        used to create new rows based on it's unique values.
    ``values`` -- column label in table for use in aggregation.
        Default None.
    ``collect`` -- aggregation function, used to group ``values``
        over row-column combinations. Default None.
    ``zero`` -- zero value to use for non-existent row-column
        combinations.

Raises:
    TypeError -- if ``collect`` is passed in and ``values`` is not,
        vice versa.

Returns:
    New pivot table, with row-col

In [10]:
# Suppose we want to look at price differences between the
# categories instead. We can use VALUES and COLLECT to do this!
# Note: you can use BOTH or NEITHER of these. Not just one!

# This table collects the cheapest option in each category.
cones.show()
cones.pivot('Flavor', 'Color', values='Price', collect = min).show()

Flavor,Color,Price
strawberry,pink,3.55
chocolate,light brown,4.75
chocolate,dark brown,5.25
strawberry,pink,5.25
chocolate,dark brown,5.25
bubblegum,pink,4.75


Color,bubblegum,chocolate,strawberry
dark brown,0,5.25,0
light brown,0,4.75,0
pink,4.75,0,3.55


In [ ]:
# Exercise: Write a function price_spread which takes in an 
# array and returns the difference between the maximum and 
# minimum in the array.
def price_spread(arr):
    ...

# Make a pivot table that has the spread of prices for each option:
# cones.pivot('Flavor', 'Color', values='Price', collect = price_spread)

## Returning to the NBA data ##

In [12]:
nba = Table.read_table('nba_salaries.csv')
nba
#nba.where('season', 2000).select('team', 'salary').group('team', sum)

rank,name,position,team,salary,season
1,Shaquille O'Neal,C,Los Angeles Lakers,17142000,2000
2,Kevin Garnett,PF,Minnesota Timberwolves,16806000,2000
3,Alonzo Mourning,C,Miami Heat,15004000,2000
4,Juwan Howard,PF,Washington Wizards,15000000,2000
5,Scottie Pippen,SF,Portland Trail Blazers,14795000,2000
6,Karl Malone,PF,Utah Jazz,14000000,2000
7,Larry Johnson,F,New York Knicks,11910000,2000
8,Gary Payton,PG,Seattle SuperSonics,11020000,2000
9,Rasheed Wallace,PF,Portland Trail Blazers,10800000,2000
10,Shawn Kemp,C,Cleveland Cavaliers,10780000,2000


In [24]:
# Exercise: For the 2000 season, use the group function to 
# find the total salary paid by each team. Repeat this for
# the 2020 season.
salary_by_team_2000 = nba.where('season', 2000).select('team', 'salary').group('team', sum)
#salary_by_team_2000
salary_by_team_2020 = nba.where('season', 2020).select('team', 'salary').group('team', sum)
#salary_by_team_2020

# Join these two
compared_salaries = salary_by_team_2000.join('team',          salary_by_team_2020,     'team')
#                    Match rows in here    using this col     with rows here    using this col 
compared_salaries
# Add a column where we look at the ratio
compared_salaries = compared_salaries.with_column(
    'Ratio', compared_salaries.column('salary sum_2') / compared_salaries.column('salary sum'))

compared_salaries.sort('Ratio', descending=True).show()
# Drop everything but the ratio and sort the table by the ratio. 
# Which team had the highest increase in salaries? The lowest?

team,salary sum,salary sum_2,Ratio
Chicago Bulls,4454000,133127813,29.8895
San Antonio Spurs,6253000,117647725,18.8146
Dallas Mavericks,9003000,134875092,14.9811
Toronto Raptors,8892000,127087525,14.2923
Houston Rockets,9593000,136992972,14.2805
Orlando Magic,10658000,142569592,13.3768
Sacramento Kings,13116000,143263849,10.9228
Detroit Pistons,15941000,135926827,8.52687
Indiana Pacers,16356000,137759737,8.42258
Washington Wizards,24375000,148744514,6.10234
